In [74]:
from bs4 import BeautifulSoup

In [139]:
import urllib3
import requests
import pandas as pd
# get source code of the page
def get_url(url):
    return urllib3.urlopen(url).read()

# makes the source tree format like 
def beautify(url):
    source = get_url(url)
    return BeautifulSoup(source,"html.parser")

In [140]:
URL = 'https://www.indeed.co.in/jobs?as_and=data+analyst&as_phr=&as_any=&as_not=&as_ttl=&as_cmp=&jt=all&st=&as_src=&salary=&radius=25&l=&fromage=any&limit=50&sort=&psf=advsrch&from=advancedsearch'
page = requests.get(URL)

soup = BeautifulSoup(page.content, 'html.parser')

In [170]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script src="//d3fw5vlhllyvee.cloudfront.net/s/ad01355/en_IN.js" type="text/javascript">
  </script>
  <link href="//d3fw5vlhllyvee.cloudfront.net/s/3c8ea85/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="http://www.indeed.co.in/rss?q=data+analyst" rel="alternate" title="Data Analyst Jobs, Recruitment" type="application/rss+xml"/>
  <link href="/m/jobs?q=data+analyst&amp;limit=50" media="only screen and (max-width: 640px)" rel="alternate"/>
  <link href="/m/jobs?q=data+analyst&amp;limit=50" media="handheld" rel="alternate"/>
  <script type="text/javascript">
   if (typeof window['closureReadyCallbacks'] == 'undefined') {
window['closureReadyCallbacks'] = [];
}

function call_when_jsall_loaded(cb) {
if (window['closureReady']) {
cb();
} else {
window['closureReadyCallbacks'].push(cb);
}
}
  </script>
  <meta content="1" name="ppstriptst"/>
  <scr

limit_output extension: Maximum message size of 10000 exceeded with 937789 characters

In [156]:
def title(soup):
    jobs = []
    for i in soup.find_all(name="div", attrs={"class":"row"}):
        for a in i.find_all(name="a", attrs={"data-tn-element":"jobTitle"}):
              jobs.append(a["title"])
    return(jobs[:-6])

In [171]:
def comp(soup): 
    companies = []
    for i in soup.find_all(name="div", attrs={"class":"row"}):
        company = i.find_all(name="span", attrs={"class":"company"})
        if len(company) > 0:
            for i in company:
                companies.append(i.text.strip())
        else:
            b = i.find_all(name="span", attrs={"class":"result-link-source"})
            for j in b:
                companies.append(j.text.strip())
    return(companies[:-6])

In [172]:
def loc(soup): 
    locations = []
    spans = soup.findAll('span', attrs={'class': 'location'})
    for i in spans:
        locations.append(i.text)
    return(locations)

In [173]:
def sal(soup): 
    salaries = []
    for i in soup.find_all(name='div', attrs={'class':'row'}):
        sal = i.find(name='div', attrs={'class':'salarySnippet'})
        salaries.append(sal.text.strip() if sal else 'Nil')
    return(salaries[:-6])

In [174]:
jobs = title(soup)
companies=comp(soup)
locations=loc(soup)
salary=sal(soup)

In [179]:
job_list=pd.DataFrame({'Job Title':jobs, 'Company':companies,'Location':locations,'Salary':salary})

In [176]:
job_list=job_list.to_excel("job_list.xls")

In [181]:
job_list

,Job Title,Company,Location,Salary
0,Analyst - Analytics,Colgate-Palmolive,"Ahmedabad, Gujarat",Nil
1,Data Analyst,EW Technologies,"Pune, Maharashtra","₹50,000 - ₹60,000 a month"
2,Systems Analyst,ICL Systems India,Remote,"₹10,00,000 a year"
3,Financial Research Analyst,DATAMATRIX TECHNOLOGIES PVT LTD,"New Delhi, Delhi","₹25,000 - ₹50,000 a month"
4,Data Analyst,56N Software Pvt Ltd,"Bengaluru, Karnataka","₹3,00,000 - ₹6,50,000 a year"
5,Data Analyst,QuinStreet Software,"Bengaluru, Karnataka","₹7,00,000 - ₹10,00,000 a year"
6,Marketing Data Analyst - Data Visualization & ...,Show Me Leads,"Raipur, Chhattisgarh",Nil
7,Data Analyst,Aerospace Asset Trading,"Indore, Madhya Pradesh",Nil
8,Data Analyst,JLL,"Thiruvananthapuram, Kerala",Nil
9,Data Analyst,Catori Infotech Private Limited,"Nashik, Maharashtra","₹2,00,000 - ₹3,00,000 a year"
